<small>Copyright 2025 Amazon.com, Inc. or its affiliates. All Rights Reserved.<br>
This is AWS Content subject to the terms of the Customer Agreement</small>

# Module 3.1: Content Alignment Evaluation Through Q&A

This notebook demonstrates the video evaluation pipeline using Amazon Nova Premier to assess video-text alignment across multiple focus areas. We'll explore how to generate structured Q&A pairs and evaluate video content against text prompts.

## A. Overview

The video evaluation pipeline consists of:
1. **Q&A Generation** - Create structured questions based on video prompts
2. **Video Analysis** - Use multimodal AI to answer questions about videos
3. **Alignment Scoring** - Calculate alignment scores across focus areas
4. **Complete Pipeline** - End-to-end evaluation with S3 storage

## B. Install Dependencies

First, let's install the required packages for this notebook.

In [ ]:
!pip install -q matplotlib opencv-python Pillow tqdm

## C. Setup and Imports

In [ ]:
import boto3
import json
import time
from botocore.exceptions import ClientError

from utils.content_alignment import generate_qa_alignment, evaluate_video_qa, evaluation_pipeline
from utils.config import get_s3_bucket, discover_video_files

## D. Configuration

Configure your AWS session and evaluation parameters. Make sure your video files and corresponding prompt files exist in S3.

- S3 bucket and video files will be automatically detected
- The notebook will use the first available video with a corresponding prompt file

In [ ]:
# AWS Configuration
session = boto3.Session()

# Get S3 bucket name
S3_BUCKET = get_s3_bucket(session)

# Load configuration for video prefix
with open('config.json', 'r') as f:
    config = json.load(f)

# Discover available videos
available_videos = discover_video_files(
    session, 
    S3_BUCKET, 
    config['video_prefix']
)

if available_videos:
    print(f"📹 Found {len(available_videos)} video(s) with prompts:")
    for i, video in enumerate(available_videos, 1):
        print(f"   {i}. {video}")
    
    # Use the first available video
    VIDEO_NAME = available_videos[0]
    print(f"\n🎯 Using video: {VIDEO_NAME}")
else:
    print("❌ No videos with prompts found. Please generate videos first.")
    VIDEO_NAME = "example.mp4"  # Fallback

S3_VIDEO_URI = f"s3://{S3_BUCKET}/{config['video_prefix']}{VIDEO_NAME}"
MODEL_ID = "us.amazon.nova-premier-v1:0"

# Focus areas for evaluation
FOCUS_AREAS = [
    "subject_alignment",
    "background_alignment", 
    "color_accuracy",
    "activity_alignment",
    "spatial_relationships"
]

## E. Step 1 - Generate Q&A Pairs

The first step creates structured questions and answers based on the video prompt and a specific focus area. The `generate_qa_alignment` function:

**Key Operations:**
1. **Atomic decomposition** - Breaks down video descriptions into smallest meaningful units
2. **Focus filtering** - Keeps only tuples relevant to the specified focus area
3. **Question generation** - Creates 5 targeted questions with multiple choice answers
4. **Answer positioning** - Provides correct answers for evaluation scoring

**Why this matters:** Structured Q&A pairs enable objective, measurable evaluation of video-text alignment across specific aspects like subjects, backgrounds, colors, activities, and spatial relationships.

In [ ]:
# Read video prompt from S3
s3_client = session.client('s3')

# Extract prompt file path from video URI
prompt_uri = S3_VIDEO_URI.replace('.mp4', '_prompt.txt')
bucket = S3_VIDEO_URI.split('/')[2]
prompt_key = '/'.join(prompt_uri.split('/')[3:])

try:
    response = s3_client.get_object(Bucket=bucket, Key=prompt_key)
    video_prompt = response['Body'].read().decode('utf-8')
    print(f"📝 Video prompt: {video_prompt}")
except Exception as e:
    print(f"❌ Error reading prompt file: {e}")
    video_prompt = "A cat playing with a ball of yarn in a cozy living room"  # Fallback

In [ ]:
# Generate Q&A pairs for subject alignment
focus_area = "background_alignment"
print(f"🎯 Generating Q&A pairs for: {focus_area}")

qa_data = generate_qa_alignment(
    boto3_session=session,
    video_prompt=video_prompt,
    focus_area=focus_area,
    model_id=MODEL_ID
)

if qa_data:
    print(f"\n📋 Generated {len(qa_data.get('questions', []))} questions")
    print(f"🧩 Atomic tuples: {json.dumps(qa_data.get('atomic_tuples', []), indent = 2)}")
    
    # Display first question as example
    if qa_data.get('questions'):
        first_q = qa_data['questions'][0]
        print(f"\n❓ Example Question: {first_q['question']}")
        print(f"📝 Answer Choices: {first_q['answer_choices']}")
        print(f"✅ Correct Answer: {first_q['correct_answer'][0]}")

        sec_q = qa_data['questions'][1]
        print(f"\n❓ Example Question: {sec_q['question']}")
        print(f"📝 Answer Choices: {sec_q['answer_choices']}")
        print(f"✅ Correct Answer: {sec_q['correct_answer'][0]}")
else:
    print("❌ Failed to generate Q&A pairs")

## F. Step 2 - Video Analysis with Multimodal AI

Now we use Amazon Nova Premier's multimodal capabilities to analyze the actual video content and answer our generated questions. The `evaluate_video_qa` function:

**Key Operations:**
1. **Multimodal input** - Processes both video content and text questions simultaneously
2. **Visual analysis** - Examines video frames to understand content
3. **Answer selection** - Chooses from provided multiple choice options
4. **Fallback handling** - Returns "None" when uncertain

**Why this matters:** This step bridges the gap between text descriptions and actual video content, enabling objective measurement of how well the generated video matches the intended prompt.

In [ ]:
# Test video analysis with the first question
if qa_data and qa_data.get('questions'):
    test_question = qa_data['questions'][0]
    question = test_question['question']
    choices = test_question['answer_choices']
    correct_answer = test_question['correct_answer'][0]
    
    print(f"🎬 Analyzing video with question: {question}")
    print(f"📋 Choices: {choices}")
    
    model_answer = evaluate_video_qa(
        session,
        s3_video_uri=S3_VIDEO_URI,
        question=question,
        answer_choices=choices,
        model_id=MODEL_ID
    )
    
    print(f"\n🤖 Model's answer: {model_answer}")
    print(f"✅ Correct answer: {correct_answer}")
    print(f"🎯 Match: {'✅ Yes' if model_answer.strip().lower() == correct_answer.lower() else '❌ No'}")
else:
    print("❌ No questions available for testing")

## G. Step 3 - Calculate Alignment Scores

This step evaluates all questions for a specific focus area and calculates an alignment score. We iterate through each question, get the model's answer, and compare it with the correct answer to generate a numerical score.

**Key Operations:**
1. **Question iteration** - Process all 5 questions for the focus area
2. **Answer comparison** - Match model responses with correct answers
3. **Score calculation** - Count correct answers out of total questions
4. **Progress tracking** - Show evaluation progress

**Scoring:** Each focus area receives a score from 0-5, representing the number of correctly answered questions.

In [ ]:
# Calculate alignment score for the focus area
if qa_data and qa_data.get('questions'):
    questions = qa_data['questions']
    score = 0
    results = []
    
    print(f"📊 Evaluating {len(questions)} questions for {focus_area}...")
    
    for i, q_data in enumerate(questions):
        question = q_data['question']
        answer_choices = q_data['answer_choices']
        correct_answer = q_data['correct_answer'][0]
        
        print(f"\n❓ Question {i+1}: {question}")
        
        # Get model's answer
        model_answer = evaluate_video_qa(
            session,
            S3_VIDEO_URI,
            question=question,
            answer_choices=answer_choices
        )
        
        # Check if answer is correct
        is_correct = model_answer.strip().lower() == correct_answer.lower()
        if is_correct:
            score += 1
            
        results.append({
            'question': question,
            'model_answer': model_answer,
            'correct_answer': correct_answer,
            'is_correct': is_correct
        })
        
        print(f"🤖 Model: {model_answer}")
        print(f"✅ Correct: {correct_answer}")
        print(f"🎯 Result: {'✅ Correct' if is_correct else '❌ Incorrect'}")
    
    print(f"\n📈 Final Score for {focus_area}: {score}/{len(questions)} ({score/len(questions)*100:.1f}%)")
else:
    print("❌ No questions available for scoring")

## H. Complete Evaluation Pipeline

The `evaluation_pipeline` function demonstrates the complete evaluation workflow across all focus areas. This wrapper function:

**Key Operations:**
1. **Multi-focus evaluation** - Processes all 5 focus areas automatically
2. **S3 integration** - Reads prompts and saves results to S3
3. **Data persistence** - Stores both scores and Q&A data for analysis
4. **Error handling** - Gracefully handles missing files or API failures

**Output Structure:**
```
s3://<bucket>/
└── generated_videos/
    ├── <video_id>.mp4
    ├── <video_id>_prompt.txt
    ├── <video_id>_alignment.json      # Alignment scores
    └── <video_id>_alignment_qa.json   # Q&A data
```

In [ ]:
print("🔧 Using production pipeline from utils/vid_eval.py...")

# Run the production pipeline
production_results = evaluation_pipeline(
    s3_video_uri=S3_VIDEO_URI,
    boto3_session=session,
    model_id=MODEL_ID
)

print(f"\n✅ Production pipeline completed!")
print(f"📊 Results: {json.dumps(production_results, indent=2)}")

## Conclusion

You've successfully implemented a comprehensive video evaluation system that:

1. **Generates structured Q&A pairs** based on video prompts and focus areas
2. **Analyzes videos using multimodal AI** to answer questions about visual content
3. **Calculates alignment scores** across multiple evaluation dimensions

Now you can move to the next module to see how to evaluate the quality of the generated video